In [1]:
import pandas as pd
df=pd.read_excel('Tipsy_updated_prototype_Ml.xlsx')

In [2]:
#Libraries import

import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
from sklearn import svm
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder

In [3]:
df

,Name,Price,Region,Brand,Brewery,Category,Taste Profile,SMV\n ?,Acidity\n ?,Fruity Category RPR,Cat name SMV,acidity value,description based on alcohol value,Amino category,acidity description based on RPR
0,"Yuki no Bosha “Kachou Gesseki” Morning Flower,...",$295.00,Akita prefecture,Yuki no Bosha,Saiya Brewing Company,Daiginjo,Light & Dry,3.0,1.2,Fruity,Mod Dry,Other,Medium+,medium rich,Light
1,Okunomatsu “Ihei”,$180.00,Fukushima prefecture,Okunomatsu,Okunomatsu Brewing Company,Daiginjo,Light & Dry,3.0,1.3,Fruity,Mod Dry,Moderately Low,Medium+,medium rich,Light
2,Kaika “Tobindori Shizuku” Black,$178.00,Tochigi prefecture,Kaika,Daiichi Brewing Company,Daiginjo,Light & Sweet,2.0,1.1,Fruity,Mod Sweet,Low,Medium+,light,Light
3,Suehiro “Gensai”,$170.00,Fukushima prefecture,Suehiro,Suehiro Brewing Company,Daiginjo,Light & Dry,4.0,1.3,Fruity,Mod Dry,Moderately Low,Medium+,medium rich,Light
4,Chiyomusubi “Daiginjo”,$162.00,Tottori prefecture,Chiyomusubi,Chiyomusubi Brewing Company,Daiginjo,Light & Dry,6.0,1.2,Fruity,Dry,Other,Medium+,medium rich,Light
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,WAKAZE “Yuzu”,$42.00,NaN,WAKAZE,WAKAZE France,Other,Light & Dry,-12.0,3.6,Very Savory,Very Sweet,High,Light,very rich,Other
490,Homare “Lychee” Nigori,NaN,Fukushima prefecture,Homare,Homare Brewing Company,Other,Rich & Sweet,-70.0,NaN,Mod Savory,Very Sweet,Other,Light,very rich,Very Savory
491,Heiwa “Tsuruume” Yuzu,$50.00,Wakayama prefecture,Heiwa,Heiwa Brewing Company,Other,Rich & Sweet,NaN,NaN,NaN,Other,Other,Light,very rich,Other
492,Heiwa “Tsuruume” Ichigo,$50.00,Wakayama prefecture,Heiwa,Heiwa Brewing Company,Other,Rich & Sweet,NaN,NaN,NaN,Other,Other,Light,very rich,Other


# Data Preprocessing

In [4]:
# Load your dataset
df = df

# Preprocessing
# Convert Price to float
df['Price'] = df['Price'].astype(str).str.replace('$', '').str.replace(',', '').astype(float)

# Convert SMV to float
df['SMV\n                  ?'] = df['SMV\n                  ?'].astype(float)

# One-hot encode categorical columns
categorical_columns = ['Region', 'Brand', 'Brewery', 'Category', 'Taste Profile', 'Fruity Category RPR', 'Cat name SMV', 'acidity value', 'description based on alcohol value', 'Amino category', 'acidity description based on RPR']
df_encoded = pd.get_dummies(df, columns=categorical_columns)

# Fill missing values with the mean of the column
df_encoded.fillna(0, inplace=True)

# Encode 'Name' column
encoder = LabelEncoder()
df_encoded['Name'] = encoder.fit_transform(df['Name'])

# Create a mapping from encoded values to original sake names
sake_name_mapping = pd.Series(encoder.classes_, index=df_encoded['Name'])


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10264\950037574.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Price'] = df['Price'].astype(str).str.replace('$', '').str.replace(',', '').astype(float)


# Machine learning

In [5]:
# Split the dataset into training and testing sets
X = df_encoded.drop('Name', axis=1)  #  'Name' is our target variable
y = df_encoded['Name']              

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the SVM model with a linear kernel and C=1.0 #for prototype kernel=linear and c=1.0
clf = svm.SVC(kernel='linear', C=1.0)
clf.fit(X_train, y_train)


SVC(kernel='linear')

# Weights Function

In [6]:
def get_weights(sake_names, like=True):
    # Get the indices of the sakes
    indices = [list(df['Name']).index(name) for name in sake_names] #if the user likes the sakes 'Sake A' and 'Sake B',
    # and these are the 1st and 3rd sakes in the dataframe, then indices would be [0, 2].
    
    # Get the data of the sakes
    sake_data = df_encoded.drop('Name', axis=1).iloc[indices]
    
    # Assign weights
    weights = sake_data.mean() + 1e-10  # Add a small constant to avoid division by zero
    if not like:
        weights = 1 / (weights + 1e-10)  # If the user doesn't like these sakes, assign lower weights

    # Normalize the weights to the range [0, 1]
    weights = (weights - weights.min()) / (weights.max() - weights.min())
    
    return weights

# Weights Finalizing and KNN Model for Recoomedation System

In [8]:
# User input
like_sake_names = input("Enter the names of the sakes you like, separated by commas: ").split(',')
dislike_sake_names = input("Enter the names of the sakes you don't like, separated by commas: ").split(',')


# Get the weights
like_weights = get_weights(like_sake_names, like=True)
dislike_weights = get_weights(dislike_sake_names, like=False)


# Combine the weights
weights = like_weights * dislike_weights


# Scale the features by the weights
scaled_X = X * weights


# Fill any new NaN values
scaled_X.fillna(0, inplace=True)


# Fit a k-NN model to the scaled data
knn = NearestNeighbors(metric='minkowski', p=2, algorithm='brute', n_neighbors=5, n_jobs=-1)
knn.fit(scaled_X)

def predict_sake(sake_name):
    # Convert the input and sake names to lowercase and remove leading/trailing
    
    sake_name = sake_name.lower().strip()
    lower_sake_names = pd.Series(list(sake_name_mapping.values)).str.lower().str.strip()

    if sake_name not in lower_sake_names.values:
        print(f"Sake '{sake_name}' not found in the dataset.")
        return

    sake_index = lower_sake_names[lower_sake_names == sake_name].index[0]
    sake_data = df_encoded.drop('Name', axis=1).iloc[sake_index]
    predicted_sake = clf.predict([sake_data])[0]
    print(f"Predicted Sake: {sake_name_mapping[predicted_sake]}")
    recommend_sakes_knn(predicted_sake)
    
    
    

# Recommend similar sakes using k-NN
def recommend_sakes_knn(sake_name):
    sake_index = list(df_encoded['Name']).index(sake_name)
    sake_data = df_encoded.drop('Name', axis=1).iloc[sake_index]
    
    # Find the k nearest neighbors
    distances, indices = knn.kneighbors([sake_data])
    
    # Get the names of the recommended sakes
    recommended_sakes = sake_name_mapping.iloc[indices[0]]
    
    print("Recommended Sakes:")
    for sake in recommended_sakes:
        print(sake)
        
        

# User input
sake_name = input("Enter the name of the sake: ")
predict_sake(sake_name)

Enter the names of the sakes you like, separated by commas: WAKAZE “Yuzu”,Okunomatsu “Ihei”
Enter the names of the sakes you don't like, separated by commas: Suehiro “Gensai”,Rihaku “Dreamy Clouds”
Enter the name of the sake: Narutotai “Ginjo” Nama Genshu


C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Predicted Sake: Narutotai “Ginjo” Nama Genshu
Recommended Sakes:
Narutotai “Ginjo” Nama Genshu
Kikumasamune “Shiboritate” Kojo
Kamotsuru “Sokaku”
Momokawa “Diamond”
Kamikokoro “Umakuchi” Hiyaoroshi


# Like Weights

In [9]:
# Print like weights
print("Like Weights:")
print(like_weights)

Like Weights:
Price                                                 1.000000
SMV\n                  ?                              0.000000
Acidity\n                  ?                          0.060173
Region_Aichi prefecture                               0.038961
Region_Akita prefecture                               0.038961
                                                        ...   
Amino category_very rich                              0.043290
acidity description based on RPR_Light                0.043290
acidity description based on RPR_Moderately Savory    0.038961
acidity description based on RPR_Other                0.043290
acidity description based on RPR_Very Savory          0.038961
Length: 446, dtype: float64


# Dislike weights

In [10]:
# Print dislike weights
print("Dislike Weights:")
print(dislike_weights)

Dislike Weights:
Price                                                 0.000000e+00
SMV\n                  ?                              5.529101e-11
Acidity\n                  ?                          1.360792e-10
Region_Aichi prefecture                               1.000000e+00
Region_Akita prefecture                               1.000000e+00
                                                          ...     
Amino category_very rich                              1.000000e+00
acidity description based on RPR_Light                3.981481e-10
acidity description based on RPR_Moderately Savory    3.981481e-10
acidity description based on RPR_Other                1.000000e+00
acidity description based on RPR_Very Savory          1.000000e+00
Length: 446, dtype: float64


# Dimensions

In [11]:
# Print dimensions used for recommendation
print("Dimensions used for recommendation:")
print(list(df_encoded.columns))

Dimensions used for recommendation:
['Name', 'Price', 'SMV\n                  ?', 'Acidity\n                  ?', 'Region_Aichi prefecture', 'Region_Akita prefecture', 'Region_Aomori prefecture', 'Region_Chiba prefecture', 'Region_Ehime prefecture', 'Region_Fukui prefecture', 'Region_Fukuoka prefecture', 'Region_Fukushima prefecture', 'Region_Gifu prefecture', 'Region_Gunma prefecture', 'Region_Hiroshima prefecture', 'Region_Hyogo prefecture', 'Region_Ibaraki prefecture', 'Region_Ishikawa prefecture', 'Region_Iwate prefecture', 'Region_Kagawa prefecture', 'Region_Kanagawa prefecture', 'Region_Kochi prefecture', 'Region_Kumamoto prefecture', 'Region_Kyoto prefecture', 'Region_Mie prefecture', 'Region_Miyagi prefecture', 'Region_Nagano prefecture', 'Region_Nara prefecture', 'Region_Niigata prefecture', 'Region_Oita prefecture', 'Region_Okayama prefecture', 'Region_Osaka prefecture', 'Region_Saga prefecture', 'Region_Shiga prefecture', 'Region_Shimane prefecture', 'Region_Shizuoka prefect

# Parameters

In [12]:
# Print parameters used for the k-NN model
print("Parameters used for the k-NN model:")
print("Metric: Minkowski")
print("Number of neighbors: 5")
print("Algorithm: Brute")

# Print parameters used for the SVM model
print("Parameters used for the SVM model:")
print("Kernel: Linear")
print("C: 1.0")

Parameters used for the k-NN model:
Metric: Minkowski
Number of neighbors: 5
Algorithm: Brute
Parameters used for the SVM model:
Kernel: Linear
C: 1.0


# Full Code 

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder

# Load your dataset
df = df

# Preprocessing
# Convert Price to float
df['Price'] = df['Price'].astype(str).str.replace('$', '').str.replace(',', '').astype(float)

# Convert SMV to float
df['SMV\n                  ?'] = df['SMV\n                  ?'].astype(float)

# One-hot encode categorical columns
categorical_columns = ['Region', 'Brand', 'Brewery', 'Category', 'Taste Profile', 'Fruity Category RPR', 'Cat name SMV', 'acidity value', 'description based on alcohol value', 'Amino category', 'acidity description based on RPR']
df_encoded = pd.get_dummies(df, columns=categorical_columns)

# Fill missing values with the mean of the column
df_encoded.fillna(0, inplace=True)

# Encode 'Name' column
encoder = LabelEncoder()
df_encoded['Name'] = encoder.fit_transform(df['Name'])

# Create a mapping from encoded values to original sake names
sake_name_mapping = pd.Series(encoder.classes_, index=df_encoded['Name'])

# Split the dataset into training and testing sets
X = df_encoded.drop('Name', axis=1)  # assuming 'Name' is your target variable
y = df_encoded['Name']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the SVM model with a linear kernel and C=1.0
clf = svm.SVC(kernel='linear', C=1.0)
clf.fit(X_train, y_train)

def get_weights(sake_names, like=True):
    # Get the indices of the sakes
    indices = [list(df['Name']).index(name) for name in sake_names]
    
    # Get the data of the sakes
    sake_data = df_encoded.drop('Name', axis=1).iloc[indices]
    
    # Assign weights
    weights = sake_data.mean() + 1e-10  # Add a small constant to avoid division by zero
    if not like:
        weights = 1 / (weights + 1e-10)  # If the user doesn't like these sakes, assign lower weights

    # Normalize the weights to the range [0, 1]
    weights = (weights - weights.min()) / (weights.max() - weights.min())
    
    return weights

# User input
like_sake_names = input("Enter the names of the sakes you like, separated by commas: ").split(',')
dislike_sake_names = input("Enter the names of the sakes you don't like, separated by commas: ").split(',')

# Get the weights
like_weights = get_weights(like_sake_names, like=True)
dislike_weights = get_weights(dislike_sake_names, like=False)

# Combine the weights
weights = like_weights * dislike_weights

# Scale the features by the weights
scaled_X = X * weights

# Fill any new NaN values
scaled_X.fillna(0, inplace=True)

# Fit a k-NN model to the scaled data
knn = NearestNeighbors(metric='minkowski', p=2, algorithm='brute', n_neighbors=5, n_jobs=-1)
knn.fit(scaled_X)

def predict_sake(sake_name):
    # Convert the input and sake names to lowercase and remove leading/trailingApologies for the abrupt cut-off. Here's the rest of the code:


    
    sake_name = sake_name.lower().strip()
    lower_sake_names = pd.Series(list(sake_name_mapping.values)).str.lower().str.strip()

    if sake_name not in lower_sake_names.values:
        print(f"Sake '{sake_name}' not found in the dataset.")
        return

    sake_index = lower_sake_names[lower_sake_names == sake_name].index[0]
    sake_data = df_encoded.drop('Name', axis=1).iloc[sake_index]
    predicted_sake = clf.predict([sake_data])[0]
    print(f"Predicted Sake: {sake_name_mapping[predicted_sake]}")
    recommend_sakes_knn(predicted_sake)

# Recommend similar sakes using k-NN
def recommend_sakes_knn(sake_name):
    sake_index = list(df_encoded['Name']).index(sake_name)
    sake_data = df_encoded.drop('Name', axis=1).iloc[sake_index]
    
    # Find the k nearest neighbors
    distances, indices = knn.kneighbors([sake_data])
    
    # Get the names of the recommended sakes
    recommended_sakes = sake_name_mapping.iloc[indices[0]]
    
    print("Recommended Sakes:")
    for sake in recommended_sakes:
        print(sake)

# User input
sake_name = input("Enter the name of the sake: ")
predict_sake(sake_name)